<a href="https://colab.research.google.com/github/ProEarth/Statistics/blob/main/Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/ExpHS300

/content/drive/MyDrive/ExpHS300


In [3]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [4]:
root = "ExpHS300"
years = ['2018']  # 需要批量就把年份列表扩展
print('years:', years)
for y in years:
  in_dir  = os.path.join(y)
  print(in_dir)
# 当前默认路径


years: ['2018']
2018


In [5]:
os.makedirs(in_dir, exist_ok=True)

cov_path = os.path.join(in_dir, 'YEAR_COV_MATRIX.xlsx')
mv_path  = os.path.join(in_dir, 'YEAR_MEAN_VAR.xlsx')
print(cov_path)
print(mv_path)
if not os.path.exists(cov_path) or not os.path.exists(mv_path):
    raise FileNotFoundError("找不到输入文件，请确认路径中包含 YEAR_COV_MATRIX.xlsx 与 YEAR_MEAN_VAR.xlsx")

# 读取
cov = pd.read_excel(cov_path, index_col=0)
mv = pd.read_excel(mv_path,  index_col=0)
print(cov)
print(mv)

2018/YEAR_COV_MATRIX.xlsx
2018/YEAR_MEAN_VAR.xlsx
               1         2         4         5         6         7         8  \
1       4.535431  3.730117  0.088126  1.683785  2.236630  0.960117  0.984999   
2       3.730117  7.181717  0.146358  2.101776  2.942103  1.672684  0.725868   
4       0.088126  0.146358  3.435273  0.807178  0.920988  0.945150  0.354565   
5       1.683785  2.101776  0.807178  5.205338  3.188975  2.611790  0.935925   
6       2.236630  2.942103  0.920988  3.188975  6.080228  2.858541  0.786647   
...          ...       ...       ...       ...       ...       ...       ...   
603993  2.816789  3.310698  0.840948  3.146913  3.570762  2.032004  0.969182   
603996  1.295078  1.677418  0.699434  2.358211  2.590597  2.061971  1.122172   
603997  0.417136  0.896821  0.467820  1.204451  0.587639  0.816426  1.156901   
603998  1.618399  2.308008  0.768518  2.708261  2.722322  2.018428  1.048530   
603999  2.046755  2.631105  1.546406  3.348920  3.074646  2.636994  1.

In [43]:
print(cov)
print(np.diag(cov.values))
std = np.sqrt(np.diag(cov.values))
corr = cov.div(std, axis=0).div(std, axis=1)
np.fill_diagonal(corr.values, 1.0)

               1         2         4         5         6         7         8  \
1       4.535431  3.730117  0.088126  1.683785  2.236630  0.960117  0.984999   
2       3.730117  7.181717  0.146358  2.101776  2.942103  1.672684  0.725868   
4       0.088126  0.146358  3.435273  0.807178  0.920988  0.945150  0.354565   
5       1.683785  2.101776  0.807178  5.205338  3.188975  2.611790  0.935925   
6       2.236630  2.942103  0.920988  3.188975  6.080228  2.858541  0.786647   
...          ...       ...       ...       ...       ...       ...       ...   
603993  2.816789  3.310698  0.840948  3.146913  3.570762  2.032004  0.969182   
603996  1.295078  1.677418  0.699434  2.358211  2.590597  2.061971  1.122172   
603997  0.417136  0.896821  0.467820  1.204451  0.587639  0.816426  1.156901   
603998  1.618399  2.308008  0.768518  2.708261  2.722322  2.018428  1.048530   
603999  2.046755  2.631105  1.546406  3.348920  3.074646  2.636994  1.347427   

               9        10        11  .

In [44]:
print(corr)
print(mv)

               1         2         4         5         6         7         8  \
1       1.000000  0.653580  0.022326  0.346540  0.425917  0.123170  0.202717   
2       0.653580  1.000000  0.029466  0.343754  0.445229  0.170526  0.118716   
4       0.022326  0.029466  1.000000  0.190882  0.201518  0.139319  0.083845   
5       0.346540  0.343754  0.190882  1.000000  0.566848  0.312756  0.179796   
6       0.425917  0.445229  0.201518  0.566848  1.000000  0.316721  0.139825   
...          ...       ...       ...       ...       ...       ...       ...   
603993  0.423379  0.395448  0.145236  0.441514  0.463538  0.177705  0.135973   
603996  0.195412  0.201137  0.121264  0.332141  0.337601  0.181025  0.158048   
603997  0.101426  0.173289  0.130701  0.273366  0.123404  0.115502  0.262567   
603998  0.290210  0.328896  0.158347  0.453317  0.421614  0.210591  0.175501   
603999  0.319442  0.326332  0.277318  0.487883  0.414449  0.239462  0.196293   

               9        10        11  .

In [45]:
class GetNetwork:
    def __init__(self,mv,corr):
        self.mv = mv
        self.df = corr

    def df2net(self, net_df):
      col_list = list(net_df.columns)
      net_df = net_df.copy()
      net_df.index = col_list

      g = nx.Graph()
      for i in range(len(col_list)):
        for j in range(i + 1, len(col_list)):   # 只看上三角
          w = net_df.iat[i, j]
          if pd.notna(w) and w != 0:
            g.add_edge(col_list[i], col_list[j], weight=float(w))
      return g


    def net2df(self, ntx):
      nodelist = list(ntx.nodes)
      # 等价但更安全：直接得到 pandas DataFrame，列行顺序一致
      df = nx.to_pandas_adjacency(ntx, nodelist=nodelist, dtype=float, weight='weight')
      return df

    def fully_connected_weight(self):
        return (1 - self.df) / 2

    def minimum_spanning_tree(self, input_graph):
        # 最小生成树Kruskal算法
        # 现在假设一个图有m个节点，n条边。
        # 首先，需要把m个节点看成m个独立的生成树，并且把n条边按照从小到大的数据进行排列。
        # 在n条边中，依次取出其中的每一条边，如果发现边的两个节点分别位于两棵树上，那么把两棵树合并成为一颗树；
        # 如果树的两个节点位于同一棵树上，那么忽略这条边，继续运行。
        # 等到所有的边都遍历结束之后，如果所有的生成树可以合并成一条生成树，那么它就是我们寻找的最小生成树，反之则没有最小生成树。
        print('计算最小生成树……')
        mst_network = nx.minimum_spanning_tree(input_graph, algorithm='kruskal')
        return mst_network

    def length(self, g2):
        length_df = pd.DataFrame(nx.floyd_warshall_numpy(g2, weight='weight'),
                                 index=g2.nodes, columns=g2.nodes)
        length_df = length_df[length_df.index]
        length_df = length_df.reindex(index=length_df.index)

        return length_df

    def neighbor(self, network_df):
        cols = network_df.columns.values
        mask = network_df.gt(0.0).values
        out = [cols[x].tolist() for x in mask]
        return out

    def get_feature(self, network_df):

        # bet_cen = nx.betweenness_centrality(g)
        #                     deg = nx.degree(g)
        #                     deg_w = nx.degree(g, weight='weight')

        deg_w  = list(nx.degree(self.df2net(network_df), weight='weight'))
        deg_df = pd.DataFrame(deg_w, columns=['stk', 'Deg']).set_index('stk')

        # print(deg_df)

        # Calculate mean of each column and create DataFrame with correct index and columns
        mean_series = network_df.mean()
        mean_df = pd.DataFrame(mean_series, columns=['Closeness'])
        mean_df.index.name = 'stk' # Set index name for consistency

        # mean_df['div(Closeness)'] = 1 / (1 + mean_df['Closeness'])
        # for index, row in mean_df.iterrows():
            # mean_df.loc[index, 'Sqrt(ADegree)'] = np.sqrt(deg_df.loc[index, 'Deg'])
            # mean_df.loc[index, 'ADegree'] = deg_df.loc[index, 'Deg']
            # mean_df.loc[index, 'div(ADegree)'] = 1 / (np.sqrt(deg_df.loc[index, 'Deg']) + 1)
            # mean_df.loc[index, 'div(ADegree)'] = 1 / (deg_df.loc[index, 'Deg'] + 1)
            # mean_df.loc[index, 'Sqrt(div(ADegree))'] = 1 / (np.sqrt(deg_df.loc[index, 'Deg']) + 1)
            # mean_df.loc[index, '1Log(Degree+1)'] = 1 / (np.log(deg_df.loc[index, 'Deg'] + 1))
            # mean_df.loc[index, '1(1+Exp(-Degree))'] = 1 / (np.log(deg_df.loc[index, 'Deg']) + 1)

        g = self.df2net(network_df)
        for node in g.nodes:
            adiv = 0

            for n_node in g.neighbors(node):
                adiv += 1 / (1 + g.edges[node, n_node]['weight'])
                # print( node,n_node,g.edges[node, n_node]['weight'])
            mean_df.loc[node, 'WDegree'] = adiv
            # mean_df.loc[node, 'Sqrt(WDegree)'] = np.sqrt(adiv)
            # mean_df.loc[node, 'Sqrt(div(WDegree))'] = 1 / (1 + np.sqrt(adiv))
            # mean_df.loc[node, 'div(WDegree)'] = 1 / (1 + adiv)

        mm_df = self.mv.copy()
        print('get feature')
        mean_df['E[return]'] = mm_df['mean']
        # mean_df['divE[return]'] = 1/(1+mm_df['mean'])
        print(mean_df)
        print(mean_df.columns)
        # -----------------------------------
                # --- 用网络的列名，避免 self.stk_list 带来的问题 ---
        cols = list(network_df.columns)

        # --- 向量化计算 pairwise mean squared difference ---
        # M: (n_rows x n_nodes) 的邻接矩阵（这里是 MST 的加权邻接）
        M = network_df.values.astype(float)

        # E[x^2]，按行求平均（列向量）
        S = (M ** 2).mean(axis=0)                    # shape: (n_nodes,)

        # E[xy]，按行求平均：E[xy] = (M^T M) / n_rows
        n_rows = M.shape[0]
        C = (M.T @ M) / n_rows                       # shape: (n_nodes, n_nodes)

        # E[(x - y)^2] = E[x^2] + E[y^2] - 2E[xy]
        msd = S[:, None] + S[None, :] - 2.0 * C      # (n_nodes, n_nodes)

        # 你的做法：str_df = 1 / (1 + str_df)
        str_mat = 1.0 / (1.0 + msd)

        str_df = pd.DataFrame(str_mat, index=cols, columns=cols)

        # （可选）如果你希望对角线是 E[x^2] 的同样转换：
        # np.fill_diagonal(str_df.values, 1.0 / (1.0 + S))
        # 否则保持当前推导也可行

        return mean_df, str_df

    def auto(self):
        # 全联通网络
        fcw = self.fully_connected_weight()

        # fcw.to_excel(os.path.join(self.path, 'B-FCN_WEIGHT.xlsx'))
        # fcw_length = self.length(self.df2net(fcw))
        # fcw_length.to_excel(os.path.join(self.path, 'B-FCN_LENGTH.xlsx'))
        # 最小生成树

        mst = self.minimum_spanning_tree(self.df2net(fcw))
        mst_df = self.net2df(mst)
        mst_df.to_excel(os.path.join(self.path, 'B-MST_WEIGHT.xlsx'))
        mst_length_df = self.length(mst)
        print('mst length')
        print(mst_length_df)

        mst_length_df.to_excel(os.path.join(self.path, 'B-MST_LENGTH.xlsx'))

        # fcn 1order 2order
        # average length
        # fcw_f1, fcw_f2 = self.get_feature(fcw_length)
        # fcw_f1.to_excel(os.path.join(self.path, 'B-FCN_FEATURE_1.xlsx'))
        # fcw_f2.to_excel(os.path.join(self.path, 'B-FCN_FEATURE_2.xlsx'))
        # mean_df = np.mean()

        # mst 1order 2order
        mst_f1, mst_f2 = self.get_feature(mst_df)
        mst_f1.to_excel(os.path.join(self.path, 'B-MST_FEATURE_1.xlsx'))
        mst_f2.to_excel(os.path.join(self.path, 'B-MST_FEATURE_2.xlsx'))

In [46]:
# 方式2：直接使用已经计算好的 corr 和 mv
# 实例化 GetNetwork 类
# Make sure in_dir is defined
out_dir = in_dir # Or define a new output directory if needed
get_network = GetNetwork(mv, corr)

# Set the path attribute for saving files
get_network.path = out_dir
get_network.stk_list = corr.columns.tolist()


# 调用 auto 方法生成网络输出
get_network.auto()

计算最小生成树……
mst length
               1         2         4         5         6         7         8  \
1       0.000000  1.563187  1.379960  1.003388  1.185569  2.104018  1.928958   
2       1.563187  0.000000  1.553492  1.176920  0.926507  2.277550  2.102489   
4       1.379960  1.553492  0.000000  0.840136  1.175875  1.783950  1.765706   
5       1.003388  1.176920  0.840136  0.000000  0.799303  1.564194  1.389134   
6       1.185569  0.926507  1.175875  0.799303  0.000000  1.899932  1.724872   
...          ...       ...       ...       ...       ...       ...       ...   
603993  1.767052  1.940584  1.603800  1.227228  1.562967  2.327858  1.832640   
603996  1.557047  1.730579  1.236979  1.017223  1.352962  1.961037  1.942793   
603997  1.600244  1.773775  1.280175  1.060420  1.396158  2.004233  1.985989   
603998  1.257510  1.431042  0.937442  0.717686  1.053425  1.661500  1.643256   
603999  1.189408  1.362940  0.869340  0.649584  0.985322  1.593397  1.575154   

               9  

In [58]:
# === 文件路径（按你的实际路径改） ===
MST_PATH = os.path.join(str(years[0]), 'B-MST_FEATURE_1.xlsx')  # or '/content/drive/MyDrive/data/B-MST_FEATURE_1.xlsx'
FIN_PATH = os.path.join('test1', 'yingli.xlsx')           # or '/content/drive/MyDrive/data/yingli.xlsx'

print(MST_PATH, FIN_PATH)
# 想要生成的年份（自行增删）

# === 读取 MST 特征 ===
# 已知工作表名：B-MST_FEATURE_1.xlsx 里是 'Sheet1'
mst_raw = pd.read_excel(MST_PATH, sheet_name='Sheet1', header=0)

# 统一列名：stk -> Stkcd；WDegree -> Deg；Closeness -> Clo
mst = mst_raw.rename(columns={'stk':'Stkcd', 'WDegree':'Deg', 'Closeness':'Clo'})

# 证券代码清洗（仅保留字母数字，转大写）
mst['Stkcd'] = (mst['Stkcd'].astype(str)
                .str.strip()
                .str.replace(r'[^0-9A-Za-z]', '', regex=True)
                .str.upper())

# 数值列转型
for c in ['Deg', 'Clo']:
    if c in mst.columns:
        mst[c] = pd.to_numeric(mst[c], errors='coerce')

mst = mst.dropna(subset=['Stkcd']).drop_duplicates(subset=['Stkcd'])

# === 读取财务（含 ROE 等）===
# 已知工作表名：yingli.xlsx 里是 'sheet1'（注意小写）
fin = pd.read_excel(FIN_PATH, sheet_name='sheet1', header=0)

# 证券代码清洗
fin['Stkcd'] = (fin['Stkcd'].astype(str)
                .str.strip()
                .str.replace(r'[^0-9A-Za-z]', '', regex=True)
                .str.upper())

# 提取年份（从 Accper 中抽取前4位数字）
year_series = fin['Accper'].astype(str).str.extract(r'(\d{4})', expand=False)
fin['Year'] = pd.to_numeric(year_series, errors='coerce')

# ROE 列（用户指定代码：F050503B）
fin['ROE'] = pd.to_numeric(fin['F050503B'], errors='coerce')

# 仅保留需要的列
keep_cols = ['Stkcd', 'Year', 'ROE']
if 'OperatingRevenue' in fin.columns:
    keep_cols.append('OperatingRevenue')
fin = fin[keep_cols]
print(fin)
# === 按年份合并并导出 ===
outputs = []
for y in years:
    sub_fin = fin.loc[fin['Year'] == int(y)].copy()
    print(sub_fin)
    # 同一公司当年可能多条，取均值聚合
    agg_cols = ['ROE'] + (['OperatingRevenue'] if 'OperatingRevenue' in sub_fin.columns else [])
    sub_fin = sub_fin.groupby('Stkcd', as_index=False)[agg_cols].mean()

    # 合并中心性 + 财务
    merged = mst.merge(sub_fin, on='Stkcd', how='inner')

    # 导出
    out_path = f'merged_{y}.csv'
    merged.to_csv(out_path, index=False)
    outputs.append((y, out_path, merged.shape))

outputs

2018/B-MST_FEATURE_1.xlsx test1/yingli.xlsx


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


         Stkcd    Year       ROE
0                  NaN       NaN
1                  NaN       NaN
2       000001  2001.0  0.096183
3       000001  2002.0  0.049452
4       000001  2002.0  0.083002
...        ...     ...       ...
529182  920819  2024.0  0.013173
529183  920819  2024.0 -0.036395
529184  920819  2024.0  0.003763
529185  920819  2024.0 -0.110117
529186  920819  2024.0 -0.002876

[529187 rows x 3 columns]
         Stkcd    Year       ROE
73      000001  2018.0  0.029611
74      000001  2018.0  0.059405
75      000001  2018.0  0.089467
76      000001  2018.0  0.107415
242     000002  2018.0  0.009316
...        ...     ...       ...
529134  920819  2018.0  0.042508
529135  920819  2018.0  0.089643
529136  920819  2018.0  0.056182
529137  920819  2018.0  0.109297
529138  920819  2018.0  0.101713

[32838 rows x 3 columns]


[('2018', 'merged_2018.csv', (2183, 5))]

In [59]:
merged_df = pd.read_csv('merged_2018.csv')
print(merged_df)


       Stkcd       Clo       Deg  E[return]       ROE
0     300001  0.000057  0.832074        NaN  0.026388
1     300002  0.000038  0.881496        NaN -0.021804
2     300003  0.000041  0.872053        NaN  0.101420
3     300004  0.000059  0.826751        NaN -0.091088
4     300005  0.000150  2.546240        NaN -0.005475
...      ...       ...       ...        ...       ...
2178  603993  0.000097  2.689089        NaN  0.057770
2179  603996  0.000129  1.625822        NaN  0.018433
2180  603997  0.000074  0.790618        NaN  0.109818
2181  603998  0.000055  0.835713        NaN  0.050139
2182  603999  0.000040  0.873753        NaN  0.015699

[2183 rows x 5 columns]


In [60]:
from scipy.stats import spearmanr
# ========== 读入数据 ==========

df = pd.read_csv('merged_2018.csv')  # 若路径不同替换这里

# 统一列名的大小写/别名（以防万一）
rename_map = {}
cols_lower = {c.lower(): c for c in df.columns}
if 'clo' in cols_lower: rename_map[cols_lower['clo']] = 'Clo'
if 'deg' in cols_lower: rename_map[cols_lower['deg']] = 'Deg'
if 'roe' in cols_lower: rename_map[cols_lower['roe']] = 'ROE'
df = df.rename(columns=rename_map)

# 只保留需要的列并转浮点
use = df[['Clo','Deg','ROE']].copy()
for c in ['Clo','Deg','ROE']:
    use[c] = pd.to_numeric(use[c], errors='coerce')

# 去掉缺失
use = use.dropna(subset=['Clo','Deg','ROE']).reset_index(drop=True)

# ========== Spearman + 置换(QAP)检验 ==========
# 置换检验：固定 X，随机置换 Y 的秩，计算两侧 p 值
def permute_p_value(x, y, n_perm=10000, seed=2026):
    rng = np.random.default_rng(seed)
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)

    # 有效样本掩码
    mask = ~(np.isnan(x) | np.isnan(y))
    x = x[mask]
    y = y[mask]
    n = len(x)
    if n < 3:
        return np.nan, np.nan, n

    # 观测值
    rho_obs, p_scipy = spearmanr(x, y, nan_policy='omit')

    # 置换
    cnt = 0
    for _ in range(n_perm):
        y_perm = rng.permutation(y)
        rho_perm, _ = spearmanr(x, y_perm)
        if abs(rho_perm) >= abs(rho_obs):
            cnt += 1
    p_perm = (cnt + 1) / (n_perm + 1)  # add-one 平滑

    return rho_obs, p_scipy, p_perm, n

pairs = [
    ('Clo', 'ROE', 'Closeness', 'ROE'),
    ('Deg', 'ROE', 'Degree',    'ROE'),
]

rows = []
for xcol, ycol, xname, yname in pairs:
    rho, p_scipy, p_perm, n = permute_p_value(use[xcol], use[ycol], n_perm=10000, seed=2025)
    rows.append({
        'X (MST Centrality)': xname,
        'Y (Economic Measure)': yname,
        'Spearman ρ': rho,
        'Scipy p': p_scipy,
        'Permutation p (QAP-style)': p_perm,
        'N (firms)': n
    })

out = pd.DataFrame(rows)
display(out.round(4))

# （可选）保存到文件
out.to_csv('spearman_qap_results_2018.csv', index=False)
print('Saved -> spearman_qap_results_2018.csv')

,X (MST Centrality),Y (Economic Measure),Spearman ρ,Scipy p,Permutation p (QAP-style),N (firms)
0,Closeness,ROE,-0.0628,0.0034,0.0032,2181
1,Degree,ROE,-0.0727,0.0007,0.0009,2181


Saved -> spearman_qap_results_2018.csv


In [ ]:
# 下面是实验2

In [18]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import pandas as pd

# ===== 参数 =====
YEAR = 2014
PATH_MST_SPARSE = os.path.join(str(YEAR), 'B-MST_WEIGHT.xlsx')
PATH_HLD_LIST = [
    os.path.join('test2', 'HLD_Shareholders.xlsx') ,
    os.path.join('test2', 'HLD_Shareholders1.xlsx') ,
    os.path.join('test2', 'HLD_Shareholders2.xlsx') ,
]
OUT_PATH = f"MST_pairs_holding_similarity_{YEAR}.xlsx"

# 仅使用“前十大股东”
ONLY_TOP_N = 10  # None 则不限制

# ======= 维度控制（强烈推荐打开其中至少一个） =======
# 1) 只保留“稀疏 MST”里权重（或变换后值）最显著的少量边：
KEEP_TOP_K_EDGES = 300_000        # 每年最多保留的边数；None 不限
EDGE_WEIGHT_PERCENTILE = 0.85     # 或按分位数过滤（保留最靠前 15% 的边）；设 None 不用
# 2) 每个股票只保留前 K 个“最大的股东权重”（而不是原表的前10名）：
PER_STOCK_TOPK_SHAREHOLDERS = 10  # None 不限；会覆盖 ONLY_TOP_N 的效果

# 列名候选
STKCD_ALIASES   = ["Stkcd", "stkcd", "code", "ticker", "证券代码"]
NAME_ALIASES    = ["S0301a", "shareholder", "holder", "股东名称", "股东"]
RATIO_ALIASES   = ["S0304a", "ratio", "percent", "percentage", "holding_pct", "持股比例"]
DATE_ALIASES    = ["Reptdt", "reportdate", "stat_date", "enddate", "日期", "统计截止日期"]
RANK_ALIASES    = ["S0306a", "rank", "holding_rank", "持股排名"]

EDGE_U_ALIASES  = ["i", "src", "from", "stk_i", "code_i", "node_i", "u"]
EDGE_V_ALIASES  = ["j", "dst", "to", "stk_j", "code_j", "node_j", "v"]
WEIGHT_ALIASES  = ["w", "weight", "mst", "dist", "distance", "value", "val"]

# ===== 小工具 =====
def _normalize_code(x):
    if pd.isna(x): return np.nan
    return str(x).strip()

def _extract_year(x):
    if pd.isna(x): return np.nan
    y = pd.to_datetime(x, errors="coerce")
    if pd.notna(y): return y.year
    s = str(x)
    return int(s[:4]) if len(s) >= 4 and s[:4].isdigit() else np.nan

def _pick_col(df, aliases):
    lowmap = {c.lower(): c for c in df.columns}
    for a in aliases:
        if a.lower() in lowmap: return lowmap[a.lower()]
    for c in df.columns:
        if any(a.lower() in c.lower() for a in aliases): return c
    return None

def _cosine_fast(d_small, d_large, norm_small, norm_large):
    """
    更快的余弦：只遍历较小的字典（OPTIMIZE）。
    d_small/d_large: shareholder -> weight(float32)
    norm_*: 预计算的 L2 范数（float32）
    """
    if not d_small or not d_large or norm_small == 0 or norm_large == 0:
        return np.nan
    dot = 0.0
    for k, w1 in d_small.items():
        w2 = d_large.get(k)
        if w2 is not None:
            dot += float(w1) * float(w2)
    return dot / (norm_small * norm_large) if dot > 0 else 0.0

def _common_count(d1, d2):
    if not d1 or not d2: return 0
    # 交集大小（遍历较小集合）
    if len(d1) <= len(d2):
        return sum(1 for k in d1.keys() if k in d2)
    else:
        return sum(1 for k in d2.keys() if k in d1)

# ===== 读取稀疏 MST =====
mst_raw = pd.read_excel(PATH_MST_SPARSE, sheet_name=0, engine="openpyxl")
mst_raw.columns = [str(c).strip() for c in mst_raw.columns]

def _try_edge_list(df):
    u = _pick_col(df, EDGE_U_ALIASES + STKCD_ALIASES)
    v = _pick_col(df, EDGE_V_ALIASES + STKCD_ALIASES)
    w = _pick_col(df, WEIGHT_ALIASES)
    if u is None or v is None or w is None:
        return None
    out = df[[u, v, w]].copy()
    out.columns = ["stk_i", "stk_j", "raw_weight"]
    return out

pairs_df = _try_edge_list(mst_raw)

if pairs_df is None:
    first_col = mst_raw.columns[0]
    colset = set(str(c).strip() for c in mst_raw.columns[1:])
    rowlike = mst_raw[first_col].astype(str).str.strip()
    if len(set(rowlike) & colset) >= max(3, int(0.6 * min(len(colset), len(rowlike)))):
        mat = mst_raw.set_index(first_col)
    else:
        mat = mst_raw.copy()
        if mat.shape[0] != mat.shape[1]:
            raise RuntimeError("无法识别为边表，也不是方阵稀疏矩阵。")
        mat.index = mat.columns
    mat.index = mat.index.map(_normalize_code)
    mat.columns = [ _normalize_code(c) for c in mat.columns ]
    codes = list(mat.columns)
    rows = []
    for i in range(len(codes)):
        vals = mat.iloc[i, i+1:].astype('float64')
        nonna = vals[vals.notna()]
        for j_off, v in nonna.items():
            rows.append((codes[i], j_off, float(v)))
    pairs_df = pd.DataFrame(rows, columns=["stk_i", "stk_j", "raw_weight"])

# 清洗 + 无向去重
pairs_df["stk_i"] = pairs_df["stk_i"].apply(_normalize_code)
pairs_df["stk_j"] = pairs_df["stk_j"].apply(_normalize_code)
pairs_df = pairs_df.dropna(subset=["stk_i", "stk_j", "raw_weight"])
pairs_df = pairs_df[pairs_df["stk_i"] != pairs_df["stk_j"]].copy()
key = pairs_df.apply(lambda r: tuple(sorted([r["stk_i"], r["stk_j"]])), axis=1)
pairs_df = pairs_df.loc[~key.duplicated()].copy()

# 先用原权重做筛选，再变换（OPTIMIZE）
if EDGE_WEIGHT_PERCENTILE is not None and 0 < EDGE_WEIGHT_PERCENTILE < 1:
    thr = pairs_df["raw_weight"].quantile(EDGE_WEIGHT_PERCENTILE)
    pairs_df = pairs_df[pairs_df["raw_weight"] >= thr].copy()

if KEEP_TOP_K_EDGES is not None and len(pairs_df) > KEEP_TOP_K_EDGES:
    pairs_df = pairs_df.nlargest(KEEP_TOP_K_EDGES, "raw_weight").copy()

# 变换：mst_value = 1 - 原值（用 float32 降内存，OPTIMIZE）
pairs_df["mst_value"] = (1.0 - pd.to_numeric(pairs_df["raw_weight"], errors="coerce")).astype("float32")
pairs_df = pairs_df.drop(columns=["raw_weight"]).reset_index(drop=True)

# 只保留“出现过的股票”集合（OPTIMIZE）
tickers = pd.unique(pairs_df[["stk_i", "stk_j"]].values.ravel("K"))
tickers_set = set(tickers)

# ===== 读取 HLD，仅保留涉及到的股票，且瘦身列（OPTIMIZE） =====
hld_list = []
use_cols_hint = set(STKCD_ALIASES + NAME_ALIASES + RATIO_ALIASES + DATE_ALIASES + RANK_ALIASES)
for p in PATH_HLD_LIST:
    if not os.path.exists(p):
        continue
    # 读进来后再列裁剪，避免 dtype 推断开销过大
    df = pd.read_excel(p, sheet_name=0, engine="openpyxl")
    keep = [c for c in df.columns if any(h.lower() in c.lower() for h in use_cols_hint)]
    df = df[keep].copy()
    date_col = _pick_col(df, DATE_ALIASES)
    if date_col is not None:
        df["_Year"] = df[date_col].apply(_extract_year)
        df = df[df["_Year"] == YEAR].copy()
    else:
        df["_Year"] = YEAR
    hld_list.append(df)

holder_map = {}
if hld_list:
    d2 = pd.concat(hld_list, ignore_index=True)

    col_stk   = _pick_col(d2, STKCD_ALIASES)
    col_name  = _pick_col(d2, NAME_ALIASES)
    col_ratio = _pick_col(d2, RATIO_ALIASES)
    col_rank  = _pick_col(d2, RANK_ALIASES) if ONLY_TOP_N else None

    # 关键：只保留出现在 pairs 的股票（OPTIMIZE）
    d2[col_stk] = d2[col_stk].apply(_normalize_code)
    d2 = d2[d2[col_stk].isin(tickers_set)].copy()

    if col_name is not None:
        d2[col_name] = d2[col_name].astype(str).str.strip()
    else:
        raise RuntimeError("缺少股东名称列，无法计算相似度。")

    # 若原数据无“排名”，也做“每股 topK 股东”（用权重排序，OPTIMIZE）
    if col_ratio is not None:
        r = pd.to_numeric(d2[col_ratio], errors="coerce")
        if pd.notna(r.max()) and r.max() > 1.5:
            r = r / 100.0
        d2[col_ratio] = r.astype("float32")
    else:
        d2[col_ratio] = np.nan

    if ONLY_TOP_N and col_rank is not None:
        d2[col_rank] = pd.to_numeric(d2[col_rank], errors="coerce")
        d2 = d2[d2[col_rank] <= ONLY_TOP_N].copy()

    # 覆盖式的“按权重取每股前K个股东”（比按排名更稳，OPTIMIZE）
    if PER_STOCK_TOPK_SHAREHOLDERS is not None:
        d2 = (d2
              .sort_values([col_stk, col_ratio], ascending=[True, False])
              .groupby(col_stk, as_index=False, group_keys=False)
              .head(PER_STOCK_TOPK_SHAREHOLDERS))

    # 聚合到“股票-股东唯一”，权重取最大
    agg = d2.groupby([col_stk, col_name], dropna=True, as_index=False)[col_ratio].max()

    # 预计算：每个股票的（股东→权重）字典 + L2 范数（float32，OPTIMIZE）
    for stk, sub in agg.groupby(col_stk):
        # 对缺失权重按 1 处理，且用 float32 缩内存
        w = sub[col_ratio].fillna(1.0).astype("float32").values
        nms = sub[col_name].astype(str).values
        d = {nms[i]: float(w[i]) for i in range(len(nms))}
        norm = np.sqrt(np.dot(w, w)).astype("float32")
        holder_map[stk] = (d, float(norm))

# ===== 仅计算出现在 pairs 的对子，且用更快的余弦（OPTIMIZE） =====
cos_list, com_list = [], []
get = holder_map.get
for a, b, mv in pairs_df[["stk_i", "stk_j", "mst_value"]].itertuples(index=False):
    v1 = get(a); v2 = get(b)
    if v1 is None or v2 is None:
        cos_list.append(np.nan); com_list.append(0)
        continue
    d1, n1 = v1; d2, n2 = v2
    # 遍历更小的 dict（加速）
    if len(d1) <= len(d2):
        cos = _cosine_fast(d1, d2, n1, n2)
        com = _common_count(d1, d2)
    else:
        cos = _cosine_fast(d2, d1, n2, n1)
        com = _common_count(d1, d2)
    cos_list.append(cos); com_list.append(com)

final = pairs_df[["stk_i", "stk_j", "mst_value"]].copy()
final["holding_cosine"] = cos_list
final["num_common_shareholders"] = com_list

# 大结果写盘（可开启分块写出，OPTIMIZE）
final.to_excel(OUT_PATH, index=False)
print(f"完成：{OUT_PATH} | 边数：{len(final):,} | 覆盖股票数：{len(tickers_set):,}")


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


完成：MST_pairs_holding_similarity_2014.xlsx | 边数：300,000 | 覆盖股票数：2,592


In [17]:
print(final.columns)
print(len(final))

Index(['stk_i', 'stk_j', 'mst_value', 'holding_cosine',
       'num_common_shareholders'],
      dtype='object')
300000


In [20]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from scipy.stats import spearmanr

# ========= I/O =========

OUT_PATH = 'spearman_qap_results_mst_cos_2014.csv'

# ========= 读入 & 清洗 =========
df = final

# 统一大小写/别名（以防有变）
cols_lower = {c.lower(): c for c in df.columns}
def col(name):
    # 允许大小写/下划线差异
    key = name.lower()
    return cols_lower.get(key, name)

xcol = col('mst_value')
ycol = col('holding_cosine')

use = df[[xcol, ycol]].copy()
use[xcol] = pd.to_numeric(use[xcol], errors='coerce')
use[ycol] = pd.to_numeric(use[ycol], errors='coerce')
use = use.dropna().reset_index(drop=True)

# ========= Spearman + 置换(QAP-style) =========
def permute_p_value(x, y, n_perm=10000, seed=2026):
    """
    固定 X，随机置换 Y 的秩（近似 QAP 的单调检验，适合稀疏对子列表）。
    返回：rho_obs, p_scipy(two-sided), p_perm(two-sided), n
    """
    rng = np.random.default_rng(seed)
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)

    mask = ~(np.isnan(x) | np.isnan(y))
    x = x[mask]
    y = y[mask]
    n = len(x)
    if n < 3:
        return np.nan, np.nan, np.nan, n

    rho_obs, p_scipy = spearmanr(x, y, nan_policy='omit')

    cnt = 0
    for _ in range(n_perm):
        y_perm = rng.permutation(y)
        rho_perm, _ = spearmanr(x, y_perm)
        if abs(rho_perm) >= abs(rho_obs):
            cnt += 1
    p_perm = (cnt + 1) / (n_perm + 1)  # add-one 平滑

    return rho_obs, p_scipy, p_perm, n

rho, p_scipy, p_perm, n = permute_p_value(use[xcol], use[ycol], n_perm=10000, seed=2025)

# 可选：单侧 p 值（如果你预期“距离越大，相似度越小”，即 rho < 0）
p_perm_one_sided = p_perm / 2 if rho < 0 else 1 - (p_perm / 2)

# ========= 输出表 =========
out = pd.DataFrame([{
    'X (MST value)': 'mst_value (1 - raw_weight)',
    'Y (Cosine similarity)': 'holding_cosine',
    'Spearman rho': rho,
    'SciPy p (two-sided)': p_scipy,
    'Permutation p (two-sided)': p_perm,
    'Permutation p (one-sided, rho<0)': p_perm_one_sided,
    'N (pairs)': n
}])

print(out.round(6))
out.round(6).to_csv(OUT_PATH, index=False)
print(f'Saved -> {OUT_PATH}')


                X (MST value) Y (Cosine similarity)  Spearman rho  \
0  mst_value (1 - raw_weight)        holding_cosine      0.205905   

   SciPy p (two-sided)  Permutation p (two-sided)  \
0                  0.0                     0.0001   

   Permutation p (one-sided, rho<0)  N (pairs)  
0                           0.99995        868  
Saved -> spearman_qap_results_mst_cos_2014.csv


In [ ]:
Second, we examine whether firms with more common shareholders relationships are tend to be more similarity in our network. To utilize the same test frame, we convert edges to nodes via the line-graph transformation (Whitney, 1992), which represents each original edge as a node and connects two nodes if the corresponding edges share an endpoint.